In [0]:
# Load data
cases = spark.read.option("header", True).csv("dbfs:/FileStore/covid_cases.csv")
pop = spark.read.option("header", True).csv("dbfs:/FileStore/population.csv")

# Convert types
cases = cases.withColumn("new_cases", col("new_cases").cast("int")) \
             .withColumn("date", to_date("date"))

pop = pop.withColumn("population", col("population").cast("int"))

# Join
df = cases.join(pop, on="state", how="left")

# Calculate cases per 100K people
df = df.withColumn("cases_per_100k", (col("new_cases") / col("population")) * 100000)

# Save to Delta
df.write.format("delta").mode("overwrite").save("dbfs:/mnt/data/covid_analysis")
